In [1]:
#import the necessary libraries
from os import listdir
from os.path import isfile, join
from skimage.transform import resize
import numpy as np
import cv2 as cv
import pandas as pd
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
from skimage.io import imread, imshow
from skimage import exposure
import csv
from sklearn.svm import LinearSVC
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from skimage import feature
from sklearn.metrics import confusion_matrix
import csv
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
#To return LBP histogram gradient
class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
    # store the number of points and radius
        self.numPoints = numPoints
        self.radius = radius
    def describe(self, image, eps=1e-7):
        # compute the Local Binary Pattern representation
        # of the image, and then use the LBP representation
        # to build the histogram of patterns
        lbp = feature.local_binary_pattern(image, self.numPoints, self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, self.numPoints + 3), range=(0, self.numPoints + 2))
        # normalize the histogram
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)
        # return the histogram of Local Binary Patterns
        return hist

In [3]:
#path where images are located
mypath='UFPR-Periocular/periocularCropped/'
#Reading files of images from folder path
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
images = np.empty(len(onlyfiles), dtype=object)

In [4]:
#Checking age range
with open('UFPR-Periocular/imagesInfo_csv.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    df = pd.read_csv(csv_file)
    image_file_name,age,gender = df.file_name,df.age,df.gender
    age_groups = pd.cut(df['age'], bins=[0,9,19,29,39,49,59,69,79,89,99])
    df.groupby(age_groups)
    print(pd.crosstab(age_groups, df['gender']))

gender    female  male
age                   
(9, 19]     3870  4350
(19, 29]    5760  7590
(29, 39]    1530  2580
(39, 49]    2070  1890
(49, 59]    1590  1170
(59, 69]     420   450
(69, 79]     360    30


In [14]:
mypath='UFPR-Periocular/periocularCropped/'
columns = defaultdict(list)
age_label_10_19 = []
lbp_features_10_19 = []
age_label_20_29 = []
lbp_features_20_29 = []
age_label_30_39 = []
lbp_features_30_39 = []
age_label_40_49 = []
lbp_features_40_49 = []
age_label_50_59 = []
lbp_features_50_59 = []
age_label_60_69 = []
lbp_features_60_69 = []
age_label_70_79 = []
lbp_features_70_79 = []


# initialize the local binary patterns descriptor along with
desc = LocalBinaryPatterns(24, 8)

with open('UFPR-Periocular/imagesInfo_csv.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    df = pd.read_csv(csv_file)
    image_file_name,age,gender = df.file_name,df.age,df.gender

    print(len(image_file_name))
    for i in range(len(image_file_name)):
        image = cv.imread( join(mypath,image_file_name[i]) )
        img = cv.cvtColor(image ,cv.COLOR_RGB2BGR)
        #convert the images to gray and describe it to extract lbp features
        gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        hist = desc.describe(gray)
        
        if age[i]<20:
            lbp_features_10_19.append(hist)
            if gender[i]== 'male':
                age_label_10_19.append("male")
            else:
                age_label_10_19.append("female")
        elif age[i]<30:    
            lbp_features_20_29.append(hist)
            if gender[i]== 'male':
                age_label_20_29.append("male")
            else:
                age_label_20_29.append("female")
        elif age[i]<40:    
            lbp_features_30_39.append(hist)
            if gender[i]== 'male':
                age_label_30_39.append("male")
            else:
                age_label_30_39.append("female")
        elif age[i]<50:    
            lbp_features_40_49.append(hist)
            if gender[i]== 'male':
                age_label_40_49.append("male")
            else:
                age_label_40_49.append("female")            
        elif age[i]<60:    
            lbp_features_50_59.append(hist)
            if gender[i]== 'male':
                age_label_50_59.append("male")
            else:
                age_label_50_59.append("female")
        elif age[i]<70:    
            lbp_features_60_69.append(hist)
            if gender[i]== 'male':
                age_label_60_69.append("male")
            else:
                age_label_60_69.append("female")      
        else:    
            lbp_features_70_79.append(hist)
            if gender[i]== 'male':
                age_label_70_79.append("male")
            else:
                age_label_70_79.append("female")  
                
            

33660


# Age Range 10-19

In [15]:
# Fit a SVM classifier to the data where gender=male
y =  np.array(age_label_10_19)
X =  np.array(lbp_features_10_19)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)
clf = svm.SVC().fit(x_train,y_train)
y_pred = clf.predict(x_test)
#Linear SVM
print("Age Range 10-19")
print("Overall Accuracy using Linear SVM: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')
C = 1.0

#Non-Linear RBF
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C)
rbf_svc.fit(x_train,y_train)
y_pred = rbf_svc.predict(x_test)  
print("Non-Linear RBF Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#Non-Linear Ploy
poly_svc = svm.SVC(kernel='poly', degree=3, C=C)
poly_svc.fit(x_train,y_train)
y_pred = poly_svc.predict(x_test)
print("Non-Linear Poly Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=500, alpha=0.0001,
                 solver='sgd', verbose=10,  random_state=21,tol=0.000000001)
mlp_clf.fit(x_train, y_train)
y_pred = mlp_clf.predict(x_test)
print("MLP Accuracy: "+str(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

Age Range 10-19
Overall Accuracy using Linear SVM: 0.5737226277372263
              precision    recall  f1-score   support

      female       0.65      0.22      0.33      1949
        male       0.56      0.89      0.69      2161

    accuracy                           0.57      4110
   macro avg       0.60      0.56      0.51      4110
weighted avg       0.60      0.57      0.52      4110

Accuracy for each class
[0.22165213 0.89125405]




Non-Linear RBF Accuracy: 0.5257907542579076
              precision    recall  f1-score   support

      female       0.50      0.00      0.00      1949
        male       0.53      1.00      0.69      2161

    accuracy                           0.53      4110
   macro avg       0.51      0.50      0.35      4110
weighted avg       0.51      0.53      0.36      4110

              precision    recall  f1-score   support

      female       0.50      0.00      0.00      1949
        male       0.53      1.00      0.69      2161

    accuracy    

Iteration 195, loss = 0.69070116
Iteration 196, loss = 0.69071111
Iteration 197, loss = 0.69070507
Iteration 198, loss = 0.69069492
Iteration 199, loss = 0.69069076
Iteration 200, loss = 0.69069064
Iteration 201, loss = 0.69069148
Iteration 202, loss = 0.69069200
Iteration 203, loss = 0.69069443
Iteration 204, loss = 0.69069079
Iteration 205, loss = 0.69069154
Iteration 206, loss = 0.69068738
Iteration 207, loss = 0.69069284
Iteration 208, loss = 0.69068794
Iteration 209, loss = 0.69071164
Iteration 210, loss = 0.69069930
Iteration 211, loss = 0.69068464
Iteration 212, loss = 0.69067992
Iteration 213, loss = 0.69068827
Iteration 214, loss = 0.69068180
Iteration 215, loss = 0.69067761
Iteration 216, loss = 0.69067322
Iteration 217, loss = 0.69066777
Iteration 218, loss = 0.69066824
Iteration 219, loss = 0.69066396
Iteration 220, loss = 0.69067136
Iteration 221, loss = 0.69068080
Iteration 222, loss = 0.69067277
Iteration 223, loss = 0.69067321
Iteration 224, loss = 0.69066948
Iteration 

Iteration 444, loss = 0.69033720
Iteration 445, loss = 0.69033603
Iteration 446, loss = 0.69033574
Iteration 447, loss = 0.69033738
Iteration 448, loss = 0.69032658
Iteration 449, loss = 0.69032977
Iteration 450, loss = 0.69032853
Iteration 451, loss = 0.69033748
Iteration 452, loss = 0.69032032
Iteration 453, loss = 0.69031553
Iteration 454, loss = 0.69032140
Iteration 455, loss = 0.69032263
Iteration 456, loss = 0.69031769
Iteration 457, loss = 0.69031643
Iteration 458, loss = 0.69033923
Iteration 459, loss = 0.69031839
Iteration 460, loss = 0.69031333
Iteration 461, loss = 0.69030772
Iteration 462, loss = 0.69031147
Iteration 463, loss = 0.69030839
Iteration 464, loss = 0.69032240
Iteration 465, loss = 0.69031112
Iteration 466, loss = 0.69030196
Iteration 467, loss = 0.69030823
Iteration 468, loss = 0.69030290
Iteration 469, loss = 0.69030236
Iteration 470, loss = 0.69030056
Iteration 471, loss = 0.69029367
Iteration 472, loss = 0.69030533
Iteration 473, loss = 0.69029554
Iteration 

/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-

# Age Range 20-29

In [16]:
# Fit a SVM classifier to the data where gender=male
y =  np.array(age_label_20_29)
X =  np.array(lbp_features_20_29)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)
clf = svm.SVC().fit(x_train,y_train)
y_pred = clf.predict(x_test)
#Linear SVM
print("Age Range 20-29")
print("Overall Accuracy using Linear SVM: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')
C = 1.0

#Non-Linear RBF
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C)
rbf_svc.fit(x_train,y_train)
y_pred = rbf_svc.predict(x_test)  
print("Non-Linear RBF Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#Non-Linear Ploy
poly_svc = svm.SVC(kernel='poly', degree=3, C=C)
poly_svc.fit(x_train,y_train)
y_pred = poly_svc.predict(x_test)
print("Non-Linear Poly Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=500, alpha=0.0001,
                 solver='sgd', verbose=10,  random_state=21,tol=0.000000001)
mlp_clf.fit(x_train, y_train)
y_pred = mlp_clf.predict(x_test)
print("MLP Accuracy: "+str(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

Age Range 20-29
Overall Accuracy using Linear SVM: 0.6302621722846442
              precision    recall  f1-score   support

      female       0.65      0.29      0.40      2857
        male       0.63      0.88      0.73      3818

    accuracy                           0.63      6675
   macro avg       0.64      0.59      0.57      6675
weighted avg       0.64      0.63      0.59      6675

Accuracy for each class
[0.29226461 0.88318491]




Non-Linear RBF Accuracy: 0.5725842696629213
              precision    recall  f1-score   support

      female       0.54      0.01      0.02      2857
        male       0.57      0.99      0.73      3818

    accuracy                           0.57      6675
   macro avg       0.56      0.50      0.37      6675
weighted avg       0.56      0.57      0.42      6675

              precision    recall  f1-score   support

      female       0.54      0.01      0.02      2857
        male       0.57      0.99      0.73      3818

    accuracy    

Iteration 196, loss = 0.67946331
Iteration 197, loss = 0.67941897
Iteration 198, loss = 0.67941557
Iteration 199, loss = 0.67934734
Iteration 200, loss = 0.67932436
Iteration 201, loss = 0.67928476
Iteration 202, loss = 0.67925323
Iteration 203, loss = 0.67922670
Iteration 204, loss = 0.67918747
Iteration 205, loss = 0.67917622
Iteration 206, loss = 0.67912473
Iteration 207, loss = 0.67908712
Iteration 208, loss = 0.67903856
Iteration 209, loss = 0.67904937
Iteration 210, loss = 0.67897595
Iteration 211, loss = 0.67899465
Iteration 212, loss = 0.67892302
Iteration 213, loss = 0.67886795
Iteration 214, loss = 0.67887857
Iteration 215, loss = 0.67880308
Iteration 216, loss = 0.67876954
Iteration 217, loss = 0.67876225
Iteration 218, loss = 0.67870919
Iteration 219, loss = 0.67867296
Iteration 220, loss = 0.67863833
Iteration 221, loss = 0.67863609
Iteration 222, loss = 0.67857005
Iteration 223, loss = 0.67854712
Iteration 224, loss = 0.67850356
Iteration 225, loss = 0.67847372
Iteration 

Iteration 445, loss = 0.67345337
Iteration 446, loss = 0.67341986
Iteration 447, loss = 0.67338802
Iteration 448, loss = 0.67340540
Iteration 449, loss = 0.67338651
Iteration 450, loss = 0.67337501
Iteration 451, loss = 0.67338315
Iteration 452, loss = 0.67336355
Iteration 453, loss = 0.67331433
Iteration 454, loss = 0.67332736
Iteration 455, loss = 0.67326496
Iteration 456, loss = 0.67325595
Iteration 457, loss = 0.67327730
Iteration 458, loss = 0.67329039
Iteration 459, loss = 0.67321873
Iteration 460, loss = 0.67323805
Iteration 461, loss = 0.67318030
Iteration 462, loss = 0.67320343
Iteration 463, loss = 0.67318454
Iteration 464, loss = 0.67317021
Iteration 465, loss = 0.67316098
Iteration 466, loss = 0.67311810
Iteration 467, loss = 0.67315694
Iteration 468, loss = 0.67323487
Iteration 469, loss = 0.67313608
Iteration 470, loss = 0.67311698
Iteration 471, loss = 0.67321236
Iteration 472, loss = 0.67308573
Iteration 473, loss = 0.67304598
Iteration 474, loss = 0.67307109
Iteration 

/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


# Age Range 30-39

In [17]:
# Fit a SVM classifier to the data where gender=male
y =  np.array(age_label_30_39)
X =  np.array(lbp_features_30_39)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)
clf = svm.SVC().fit(x_train,y_train)
y_pred = clf.predict(x_test)
#Linear SVM
print("Age Range 30-39")
print("Overall Accuracy using Linear SVM: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')
C = 1.0

#Non-Linear RBF
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C)
rbf_svc.fit(x_train,y_train)
y_pred = rbf_svc.predict(x_test)  
print("Non-Linear RBF Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#Non-Linear Ploy
poly_svc = svm.SVC(kernel='poly', degree=3, C=C)
poly_svc.fit(x_train,y_train)
y_pred = poly_svc.predict(x_test)
print("Non-Linear Poly Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=500, alpha=0.0001,
                 solver='sgd', verbose=10,  random_state=21,tol=0.000000001)
mlp_clf.fit(x_train, y_train)
y_pred = mlp_clf.predict(x_test)
print("MLP Accuracy: "+str(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

Age Range 30-39
Overall Accuracy using Linear SVM: 0.635036496350365
              precision    recall  f1-score   support

      female       0.83      0.03      0.06       769
        male       0.63      1.00      0.77      1286

    accuracy                           0.64      2055
   macro avg       0.73      0.51      0.42      2055
weighted avg       0.71      0.64      0.51      2055

Accuracy for each class
[0.03120936 0.99611198]




Non-Linear RBF Accuracy: 0.6257907542579075
              precision    recall  f1-score   support

      female       0.00      0.00      0.00       769
        male       0.63      1.00      0.77      1286

    accuracy                           0.63      2055
   macro avg       0.31      0.50      0.38      2055
weighted avg       0.39      0.63      0.48      2055

              precision    recall  f1-score   support

      female       0.00      0.00      0.00       769
        male       0.63      1.00      0.77      1286

    accuracy     

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metr

Non-Linear Poly Accuracy: 0.6267639902676398
              precision    recall  f1-score   support

      female       1.00      0.00      0.01       769
        male       0.63      1.00      0.77      1286

    accuracy                           0.63      2055
   macro avg       0.81      0.50      0.39      2055
weighted avg       0.77      0.63      0.48      2055

              precision    recall  f1-score   support

      female       1.00      0.00      0.01       769
        male       0.63      1.00      0.77      1286

    accuracy                           0.63      2055
   macro avg       0.81      0.50      0.39      2055
weighted avg       0.77      0.63      0.48      2055

Accuracy for each class
[0.00260078 1.        ]




Iteration 1, loss = 0.73979105
Iteration 2, loss = 0.72524904
Iteration 3, loss = 0.71089475
Iteration 4, loss = 0.69861733
Iteration 5, loss = 0.68970095
Iteration 6, loss = 0.68274370
Iteration 7, loss = 0.67764666
Iteration 8, loss = 0.67355219
I

Iteration 239, loss = 0.65660716
Iteration 240, loss = 0.65660942
Iteration 241, loss = 0.65660000
Iteration 242, loss = 0.65658869
Iteration 243, loss = 0.65658042
Iteration 244, loss = 0.65657107
Iteration 245, loss = 0.65658643
Iteration 246, loss = 0.65655252
Iteration 247, loss = 0.65655454
Iteration 248, loss = 0.65653659
Iteration 249, loss = 0.65653768
Iteration 250, loss = 0.65651280
Iteration 251, loss = 0.65650833
Iteration 252, loss = 0.65649287
Iteration 253, loss = 0.65649865
Iteration 254, loss = 0.65650394
Iteration 255, loss = 0.65648162
Iteration 256, loss = 0.65647509
Iteration 257, loss = 0.65646649
Iteration 258, loss = 0.65645022
Iteration 259, loss = 0.65644717
Iteration 260, loss = 0.65643922
Iteration 261, loss = 0.65641922
Iteration 262, loss = 0.65646474
Iteration 263, loss = 0.65640362
Iteration 264, loss = 0.65638967
Iteration 265, loss = 0.65638719
Iteration 266, loss = 0.65638635
Iteration 267, loss = 0.65637642
Iteration 268, loss = 0.65635881
Iteration 

Iteration 489, loss = 0.65376270
Iteration 490, loss = 0.65374590
Iteration 491, loss = 0.65372462
Iteration 492, loss = 0.65370489
Iteration 493, loss = 0.65370366
Iteration 494, loss = 0.65370269
Iteration 495, loss = 0.65367565
Iteration 496, loss = 0.65368109
Iteration 497, loss = 0.65363704
Iteration 498, loss = 0.65362296
Iteration 499, loss = 0.65359460
Iteration 500, loss = 0.65357049
MLP Accuracy: 0.6257907542579075
              precision    recall  f1-score   support

      female       0.00      0.00      0.00       769
        male       0.63      1.00      0.77      1286

    accuracy                           0.63      2055
   macro avg       0.31      0.50      0.38      2055
weighted avg       0.39      0.63      0.48      2055

Accuracy for each class
[0. 1.]






/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-

# Age Range 40-49

In [18]:
# Fit a SVM classifier to the data where gender=male
y =  np.array(age_label_40_49)
X =  np.array(lbp_features_40_49)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)
clf = svm.SVC().fit(x_train,y_train)
y_pred = clf.predict(x_test)
#Linear SVM
print("Age Range 40-49")
print("Overall Accuracy using Linear SVM: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')
C = 1.0

#Non-Linear RBF
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C)
rbf_svc.fit(x_train,y_train)
y_pred = rbf_svc.predict(x_test)  
print("Non-Linear RBF Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#Non-Linear Ploy
poly_svc = svm.SVC(kernel='poly', degree=3, C=C)
poly_svc.fit(x_train,y_train)
y_pred = poly_svc.predict(x_test)
print("Non-Linear Poly Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=500, alpha=0.0001,
                 solver='sgd', verbose=10,  random_state=21,tol=0.000000001)
mlp_clf.fit(x_train, y_train)
y_pred = mlp_clf.predict(x_test)
print("MLP Accuracy: "+str(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

Age Range 40-49
Overall Accuracy using Linear SVM: 0.5904040404040404
              precision    recall  f1-score   support

      female       0.60      0.64      0.62      1020
        male       0.58      0.54      0.56       960

    accuracy                           0.59      1980
   macro avg       0.59      0.59      0.59      1980
weighted avg       0.59      0.59      0.59      1980

Accuracy for each class
[0.64117647 0.53645833]




Non-Linear RBF Accuracy: 0.5151515151515151
              precision    recall  f1-score   support

      female       0.52      1.00      0.68      1020
        male       0.00      0.00      0.00       960

    accuracy                           0.52      1980
   macro avg       0.26      0.50      0.34      1980
weighted avg       0.27      0.52      0.35      1980

              precision    recall  f1-score   support

      female       0.52      1.00      0.68      1020
        male       0.00      0.00      0.00       960

    accuracy    

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metr

Non-Linear Poly Accuracy: 0.6
              precision    recall  f1-score   support

      female       0.59      0.76      0.66      1020
        male       0.63      0.43      0.51       960

    accuracy                           0.60      1980
   macro avg       0.61      0.60      0.59      1980
weighted avg       0.61      0.60      0.59      1980

              precision    recall  f1-score   support

      female       0.59      0.76      0.66      1020
        male       0.63      0.43      0.51       960

    accuracy                           0.60      1980
   macro avg       0.61      0.60      0.59      1980
weighted avg       0.61      0.60      0.59      1980

Accuracy for each class
[0.75882353 0.43125   ]




Iteration 1, loss = 0.69552723
Iteration 2, loss = 0.69476071
Iteration 3, loss = 0.69410904
Iteration 4, loss = 0.69344266
Iteration 5, loss = 0.69298576
Iteration 6, loss = 0.69259661
Iteration 7, loss = 0.69234003
Iteration 8, loss = 0.69206168
Iteration 9, los

Iteration 240, loss = 0.69049152
Iteration 241, loss = 0.69049523
Iteration 242, loss = 0.69048925
Iteration 243, loss = 0.69049464
Iteration 244, loss = 0.69048512
Iteration 245, loss = 0.69051694
Iteration 246, loss = 0.69049082
Iteration 247, loss = 0.69047731
Iteration 248, loss = 0.69047088
Iteration 249, loss = 0.69046393
Iteration 250, loss = 0.69048278
Iteration 251, loss = 0.69045922
Iteration 252, loss = 0.69045804
Iteration 253, loss = 0.69045369
Iteration 254, loss = 0.69044462
Iteration 255, loss = 0.69046635
Iteration 256, loss = 0.69043829
Iteration 257, loss = 0.69044983
Iteration 258, loss = 0.69044865
Iteration 259, loss = 0.69043731
Iteration 260, loss = 0.69043471
Iteration 261, loss = 0.69042548
Iteration 262, loss = 0.69042415
Iteration 263, loss = 0.69042013
Iteration 264, loss = 0.69041497
Iteration 265, loss = 0.69041061
Iteration 266, loss = 0.69041954
Iteration 267, loss = 0.69041533
Iteration 268, loss = 0.69040958
Iteration 269, loss = 0.69043757
Iteration 

Iteration 491, loss = 0.68954503
Iteration 492, loss = 0.68954184
Iteration 493, loss = 0.68953693
Iteration 494, loss = 0.68953645
Iteration 495, loss = 0.68954735
Iteration 496, loss = 0.68952896
Iteration 497, loss = 0.68958926
Iteration 498, loss = 0.68951440
Iteration 499, loss = 0.68951678
Iteration 500, loss = 0.68950816
MLP Accuracy: 0.5151515151515151
              precision    recall  f1-score   support

      female       0.52      1.00      0.68      1020
        male       0.00      0.00      0.00       960

    accuracy                           0.52      1980
   macro avg       0.26      0.50      0.34      1980
weighted avg       0.27      0.52      0.35      1980

Accuracy for each class
[1. 0.]






/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-

# Age Range 50-59

In [19]:
# Fit a SVM classifier to the data where gender=male
y =  np.array(age_label_50_59)
X =  np.array(lbp_features_50_59)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)
clf = svm.SVC().fit(x_train,y_train)
y_pred = clf.predict(x_test)
#Linear SVM
print("Age Range 50-59")
print("Overall Accuracy using Linear SVM: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')
C = 1.0

#Non-Linear RBF
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C)
rbf_svc.fit(x_train,y_train)
y_pred = rbf_svc.predict(x_test)  
print("Non-Linear RBF Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#Non-Linear Ploy
poly_svc = svm.SVC(kernel='poly', degree=3, C=C)
poly_svc.fit(x_train,y_train)
y_pred = poly_svc.predict(x_test)
print("Non-Linear Poly Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=500, alpha=0.0001,
                 solver='sgd', verbose=10,  random_state=21,tol=0.000000001)
mlp_clf.fit(x_train, y_train)
y_pred = mlp_clf.predict(x_test)
print("MLP Accuracy: "+str(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

Age Range 50-59
Overall Accuracy using Linear SVM: 0.6014492753623188
              precision    recall  f1-score   support

      female       0.60      0.88      0.72       787
        male       0.59      0.23      0.33       593

    accuracy                           0.60      1380
   macro avg       0.60      0.56      0.52      1380
weighted avg       0.60      0.60      0.55      1380

Accuracy for each class
[0.88182973 0.22934233]




Non-Linear RBF Accuracy: 0.5978260869565217
              precision    recall  f1-score   support

      female       0.60      0.87      0.71       787
        male       0.58      0.24      0.34       593

    accuracy                           0.60      1380
   macro avg       0.59      0.55      0.52      1380
weighted avg       0.59      0.60      0.55      1380

              precision    recall  f1-score   support

      female       0.60      0.87      0.71       787
        male       0.58      0.24      0.34       593

    accuracy    

Iteration 198, loss = 0.67631760
Iteration 199, loss = 0.67630977
Iteration 200, loss = 0.67629136
Iteration 201, loss = 0.67627722
Iteration 202, loss = 0.67623607
Iteration 203, loss = 0.67620880
Iteration 204, loss = 0.67619475
Iteration 205, loss = 0.67618265
Iteration 206, loss = 0.67615266
Iteration 207, loss = 0.67612709
Iteration 208, loss = 0.67610649
Iteration 209, loss = 0.67609869
Iteration 210, loss = 0.67606836
Iteration 211, loss = 0.67604798
Iteration 212, loss = 0.67602578
Iteration 213, loss = 0.67600231
Iteration 214, loss = 0.67598201
Iteration 215, loss = 0.67596094
Iteration 216, loss = 0.67595952
Iteration 217, loss = 0.67591600
Iteration 218, loss = 0.67590198
Iteration 219, loss = 0.67587430
Iteration 220, loss = 0.67585388
Iteration 221, loss = 0.67583596
Iteration 222, loss = 0.67580906
Iteration 223, loss = 0.67581310
Iteration 224, loss = 0.67576197
Iteration 225, loss = 0.67574200
Iteration 226, loss = 0.67571763
Iteration 227, loss = 0.67570319
Iteration 

Iteration 458, loss = 0.66847052
Iteration 459, loss = 0.66843759
Iteration 460, loss = 0.66838307
Iteration 461, loss = 0.66835576
Iteration 462, loss = 0.66829655
Iteration 463, loss = 0.66827369
Iteration 464, loss = 0.66820885
Iteration 465, loss = 0.66816702
Iteration 466, loss = 0.66811186
Iteration 467, loss = 0.66805969
Iteration 468, loss = 0.66801686
Iteration 469, loss = 0.66798673
Iteration 470, loss = 0.66792635
Iteration 471, loss = 0.66786895
Iteration 472, loss = 0.66784816
Iteration 473, loss = 0.66777600
Iteration 474, loss = 0.66773104
Iteration 475, loss = 0.66768864
Iteration 476, loss = 0.66765135
Iteration 477, loss = 0.66759459
Iteration 478, loss = 0.66756620
Iteration 479, loss = 0.66749734
Iteration 480, loss = 0.66745270
Iteration 481, loss = 0.66740222
Iteration 482, loss = 0.66734378
Iteration 483, loss = 0.66729072
Iteration 484, loss = 0.66725991
Iteration 485, loss = 0.66720585
Iteration 486, loss = 0.66717179
Iteration 487, loss = 0.66711486
Iteration 

/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-

# Age Range 60-69

In [20]:
# Fit a SVM classifier to the data where gender=male
y =  np.array(age_label_60_69)
X =  np.array(lbp_features_60_69)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)
clf = svm.SVC().fit(x_train,y_train)
y_pred = clf.predict(x_test)
#Linear SVM
print("Age Range 60-69")
print("Overall Accuracy using Linear SVM: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')
C = 1.0

#Non-Linear RBF
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C)
rbf_svc.fit(x_train,y_train)
y_pred = rbf_svc.predict(x_test)  
print("Non-Linear RBF Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#Non-Linear Ploy
poly_svc = svm.SVC(kernel='poly', degree=3, C=C)
poly_svc.fit(x_train,y_train)
y_pred = poly_svc.predict(x_test)
print("Non-Linear Poly Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=500, alpha=0.0001,
                 solver='sgd', verbose=10,  random_state=21,tol=0.000000001)
mlp_clf.fit(x_train, y_train)
y_pred = mlp_clf.predict(x_test)
print("MLP Accuracy: "+str(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

Age Range 60-69
Overall Accuracy using Linear SVM: 0.5862068965517241
              precision    recall  f1-score   support

      female       0.63      0.25      0.36       201
        male       0.58      0.87      0.69       234

    accuracy                           0.59       435
   macro avg       0.60      0.56      0.53       435
weighted avg       0.60      0.59      0.54       435

Accuracy for each class
[0.25373134 0.87179487]




Non-Linear RBF Accuracy: 0.5517241379310345
              precision    recall  f1-score   support

      female       0.52      0.47      0.49       201
        male       0.58      0.62      0.60       234

    accuracy                           0.55       435
   macro avg       0.55      0.55      0.55       435
weighted avg       0.55      0.55      0.55       435

              precision    recall  f1-score   support

      female       0.52      0.47      0.49       201
        male       0.58      0.62      0.60       234

    accuracy    

Iteration 217, loss = 0.69248916
Iteration 218, loss = 0.69248216
Iteration 219, loss = 0.69247415
Iteration 220, loss = 0.69250892
Iteration 221, loss = 0.69251460
Iteration 222, loss = 0.69250558
Iteration 223, loss = 0.69249604
Iteration 224, loss = 0.69248293
Iteration 225, loss = 0.69247317
Iteration 226, loss = 0.69246624
Iteration 227, loss = 0.69246484
Iteration 228, loss = 0.69246246
Iteration 229, loss = 0.69246782
Iteration 230, loss = 0.69245790
Iteration 231, loss = 0.69245001
Iteration 232, loss = 0.69245063
Iteration 233, loss = 0.69244806
Iteration 234, loss = 0.69243750
Iteration 235, loss = 0.69243012
Iteration 236, loss = 0.69245316
Iteration 237, loss = 0.69244731
Iteration 238, loss = 0.69245904
Iteration 239, loss = 0.69245279
Iteration 240, loss = 0.69245281
Iteration 241, loss = 0.69245313
Iteration 242, loss = 0.69245753
Iteration 243, loss = 0.69245034
Iteration 244, loss = 0.69246254
Iteration 245, loss = 0.69244513
Iteration 246, loss = 0.69244455
Training l

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Age Range 70-79

In [21]:
# Fit a SVM classifier to the data where gender=male
y =  np.array(age_label_70_79)
X =  np.array(lbp_features_70_79)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)
clf = svm.SVC().fit(x_train,y_train)
y_pred = clf.predict(x_test)
#Linear SVM
print("Age Range 70-79")
print("Overall Accuracy using Linear SVM: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')
C = 1.0

#Non-Linear RBF
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C)
rbf_svc.fit(x_train,y_train)
y_pred = rbf_svc.predict(x_test)  
print("Non-Linear RBF Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#Non-Linear Ploy
poly_svc = svm.SVC(kernel='poly', degree=3, C=C)
poly_svc.fit(x_train,y_train)
y_pred = poly_svc.predict(x_test)
print("Non-Linear Poly Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=500, alpha=0.0001,
                 solver='sgd', verbose=10,  random_state=21,tol=0.000000001)
mlp_clf.fit(x_train, y_train)
y_pred = mlp_clf.predict(x_test)
print("MLP Accuracy: "+str(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

Age Range 70-79
Overall Accuracy using Linear SVM: 0.9333333333333333
              precision    recall  f1-score   support

      female       0.93      1.00      0.97       182
        male       0.00      0.00      0.00        13

    accuracy                           0.93       195
   macro avg       0.47      0.50      0.48       195
weighted avg       0.87      0.93      0.90       195

Accuracy for each class
[1. 0.]




Non-Linear RBF Accuracy: 0.9333333333333333
              precision    recall  f1-score   support

      female       0.93      1.00      0.97       182
        male       0.00      0.00      0.00        13

    accuracy                           0.93       195
   macro avg       0.47      0.50      0.48       195
weighted avg       0.87      0.93      0.90       195

              precision    recall  f1-score   support

      female       0.93      1.00      0.97       182
        male       0.00      0.00      0.00        13

    accuracy                    

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metr

Iteration 3, loss = 0.57709143
Iteration 4, loss = 0.57494669
Iteration 5, loss = 0.57240619
Iteration 6, loss = 0.56951780
Iteration 7, loss = 0.56632333
Iteration 8, loss = 0.56286112
Iteration 9, loss = 0.55916317
Iteration 10, loss = 0.55526329
Iteration 11, loss = 0.55119034
Iteration 12, loss = 0.54697022
Iteration 13, loss = 0.54262753
Iteration 14, loss = 0.53819034
Iteration 15, loss = 0.53369718
Iteration 16, loss = 0.52918381
Iteration 17, loss = 0.52466890
Iteration 18, loss = 0.52016875
Iteration 19, loss = 0.51567053
Iteration 20, loss = 0.51118318
Iteration 21, loss = 0.50671277
Iteration 22, loss = 0.50226715
Iteration 23, loss = 0.49785904
Iteration 24, loss = 0.49349787
Iteration 25, loss = 0.48919263
Iteration 26, loss = 0.48494589
Iteration 27, loss = 0.48076213
Iteration 28, loss = 0.47664588
Iteration 29, loss = 0.47259879
Iteration 30, loss = 0.46862520
Iteration 31, loss = 0.46472564
Iteration 32, loss = 0.46090356
Iteration 33, loss = 0.45715876
Iteration 34, l

Iteration 261, loss = 0.29964677
Iteration 262, loss = 0.29962347
Iteration 263, loss = 0.29960049
Iteration 264, loss = 0.29957781
Iteration 265, loss = 0.29955538
Iteration 266, loss = 0.29953327
Iteration 267, loss = 0.29951150
Iteration 268, loss = 0.29949008
Iteration 269, loss = 0.29946900
Iteration 270, loss = 0.29944818
Iteration 271, loss = 0.29942763
Iteration 272, loss = 0.29940738
Iteration 273, loss = 0.29938746
Iteration 274, loss = 0.29936786
Iteration 275, loss = 0.29934860
Iteration 276, loss = 0.29932966
Iteration 277, loss = 0.29931103
Iteration 278, loss = 0.29929269
Iteration 279, loss = 0.29927456
Iteration 280, loss = 0.29925668
Iteration 281, loss = 0.29923902
Iteration 282, loss = 0.29922156
Iteration 283, loss = 0.29920432
Iteration 284, loss = 0.29918724
Iteration 285, loss = 0.29917041
Iteration 286, loss = 0.29915379
Iteration 287, loss = 0.29913739
Iteration 288, loss = 0.29912122
Iteration 289, loss = 0.29910528
Iteration 290, loss = 0.29908957
Iteration 

/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-